# Database creation
## Libraries import and connection establishing

In [ ]:
import sqlite3, base64

connection = sqlite3.connect('database.db')
cursor = connection.cursor()

Cell to create tables and relationships

In [ ]:
with open("database.sql", 'r') as script:
    statements = script.read().split(';')
    for statement in statements:
        cursor.execute(statement)
connection.commit()
cursor.execute("SELECT name FROM sqlite_master;")
cursor.fetchall()

In [ ]:
def getNotebook(notebook):
    if notebook.startswith('c'):
        notebook = open("../Notebooks/trainings/convnext/".join([notebook,'.ipynb']))
    elif notebook.startswith('r'):
        notebook = open("../Notebooks/trainings/resnetv2/".join([notebook,'.ipynb']))
    elif notebook.startswith('x'):
        notebook = open("../Notebooks/trainings/xception/".join([notebook,'.ipynb']))
    return notebook.read().splitlines()

def getNotebookImages(notebook):
    image_list = []
    lines = getNotebook(notebook)

    for line in reversed(lines):
        if line.startswith('"image/'):
            parts = line.split(': ')
            image_format = parts[0].strip('"')
            image_data = parts[1].strip().strip('"')

            decoded_image = base64.b64decode(image_data)

            image_list.append((image_format, decoded_image))

            if len(image_list) == 13:
                break

    return image_list

Insert information in database

In [ ]:
cursor.execute("INSERT INTO configuration (transfer_learning,freeze_layers,balance_classes,segmentation,data_augmentation,normalization,custom_optimizer,callback) VALUES (false,false,false,false,false,\"none\",false,false);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,loss_epochs,accuracy_epochs,auc_epochs,recall_epochs,precision_epochs,configuration_id) VALUES (,\"ConvNext\",?,,,,,,,,,,,,,,,,,,,,,,,,)")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,loss_epochs,accuracy_epochs,auc_epochs,recall_epochs,precision_epochs,configuration_id) VALUES (,\"ResNetV2\",?,,,,,,,,,,,,,,,,,,,,,,,,)")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,loss_epochs,accuracy_epochs,auc_epochs,recall_epochs,precision_epochs,configuration_id) VALUES (,\"Xception\",?,,,,,,,,,,,,,,,,,,,,,,,,)")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

connection.commit()

In [ ]:
cursor.execute("INSERT INTO configuration (transfer_learning,freeze_layers,balance_classes,segmentation,data_augmentation,normalization,custom_optimizer,callback) VALUES (false,false,true,false,false,\"none\",false,false);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ConvNext\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ResNetV2\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"Xception\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

connection.commit()

In [ ]:
cursor.execute("INSERT INTO configuration (transfer_learning,freeze_layers,balance_classes,segmentation,data_augmentation,normalization,custom_optimizer,callback) VALUES (true,true,true,false,false,\"StandardScaler\",false,false);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ConvNext\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ResNetV2\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"Xception\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

connection.commit()

In [ ]:
cursor.execute("INSERT INTO configuration (transfer_learning,freeze_layers,balance_classes,segmentation,data_augmentation,normalization,custom_optimizer,callback) VALUES (true,true,false,false,false,\"none\",false,false);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ConvNext\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ResNetV2\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"Xception\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

connection.commit()

In [ ]:
cursor.execute("INSERT INTO configuration (transfer_learning,freeze_layers,balance_classes,segmentation,data_augmentation,normalization,custom_optimizer,callback) VALUES (false,false,false,true,false,\"none\",false,false);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ConvNext\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ResNetV2\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"Xception\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

connection.commit()

In [ ]:
cursor.execute("INSERT INTO configuration (transfer_learning,freeze_layers,balance_classes,segmentation,data_augmentation,normalization,custom_optimizer,callback) VALUES (true,true,true,true,true,\"none\",true,false);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ConvNext\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ResNetV2\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"Xception\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

connection.commit()

In [ ]:
cursor.execute("INSERT INTO configuration (transfer_learning,freeze_layers,balance_classes,segmentation,data_augmentation,normalization,custom_optimizer,callback) VALUES (false,false,false,false,true,\"none\",false,false);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ConvNext\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"ResNetV2\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

cursor.execute("INSERT INTO noteboook (name,architecture,confusion_matrix,loss,accuracy,auc,recall,precision,false_positives,false_negatives,true_negatives,true_positives,val_loss,val_accuracy,val_auc,val_recall,val_precision,val_false_positives,val_false_negatives,val_true_negatives,val_true_positives,configuration_id) VALUES (,\"Xception\",?,,,,,,,,,,,,,,,,,,,)", image_data)
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Basal_cell_carcinoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Benign_keratosis\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Bowens_disease\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Dermatofibroma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanocytic_nevi\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Melanoma\",?,,,,,);")
cursor.execute("INSERT INTO metrics (type,confusion_matrix,precision,f1_score,roc_auc,recall,specificity,notebook_id) VALUES (\"Vascular_lesion\",?,,,,,);")

connection.commit()

Reading image files

In [ ]:
with open('/content/gdrive/MyDrive/tcc/test.jpg', 'rb') as file:
    image_data = file.read()

Test if information have been saved

In [ ]:
cursor.execute('SELECT * FROM configuration')
cursor.fetchall()

#retrieved_image_data = row[0]

#with open('retrieved_image.jpg', 'wb') as file:
#    file.write(retrieved_image_data)

Close the current connection

In [ ]:
connection.close()